In [1]:
import scrapy, os
import pandas as pd
from scrapy.crawler import CrawlerProcess
from scrapy.exporters import CsvItemExporter
from scrapy.linkextractors import LinkExtractor
from scrapy.item import DictItem, Field

In [2]:
def create_item_class(class_name,field_list):
    field_dict = {}
    for field_name in field_list:
        field_dict[field_name] = Field()

    return type(class_name, (DictItem,), {'fields':field_dict})


In [3]:
class Pipeline(object):
    """Scrapy Pipeline Object for Links, Results, Team, Individual"""
    
    def __init__(self):
        self.files = {}
        self.exporters = {}
    
    def create_file(self, file, _dir): 
        # if the file doesn't exist create it 
        if file not in self.files:
            self.files[file] = open(os.path.join(_dir, file + '.csv'), 'wb')
            self.exporters[file] = CsvItemExporter(self.files[file])
            self.exporters[file].start_exporting()
    
    def close_spider(self, spider):
        [e.finish_exporting() for e in self.exporters.values()]
        [f.close() for f in self.files.values()]
 
    def process_item(self, item, spider):
        # Define what item type it is 
        file = type(item).__name__.replace("item_", "").lower()
        if "link" in file: 
            _dir = "Links" 
        else: 
            _dir = "Data"
        
        self.create_file(file, _dir)
        
        # Export 
        self.exporters[file].export_item(item)
        
        return item

In [4]:
class StatsSpider(scrapy.Spider): 
    name = "Stats"
    regex = '.*\/stats\?id\=\d+\&year_stat_category_id\=\d+'
    gamebygameregex = '.*\/player\/game_by_game\?'
    xpath = "//body//div[@id='contentarea']//div[@id='stats_div']//table"
    
    def start_requests(self): 
        file_in = 'teaminfolinks.csv'

        # Pull in the data 
        data = pd.read_csv(os.path.join("Links", file_in))
        stats = data[data.key == 'stats'].drop_duplicates()
        stats.sort_values(['team'], inplace = True)
        stats_list = stats.to_dict(orient='record')
        
        # Loop through the links 
        for url in stats_list[:1]: 
            yield scrapy.Request(url=url['link'], 
                                 callback=self.parse,
                                 meta = {'team':url['team']}
                                )
    
    def parse(self, response): 
        # Year 
        year = response.selector.xpath("//body//div[@id='contentarea']//fieldset//div/" +
                 "/form[@id='change_sport_form']//select[@id='year_list']//option[@select" +
                 "ed='selected']//text()").extract()[0]
        
        # Write out the rushing stats 
        table = self.table_cleaner(html = response.body, 
                           target_table = 2, 
                           stat = 'rushing', 
                           year = year, 
                           team = response.meta['team'],
                           trim_n_rows = 1
                          )
        
        # Create dynamic items
        stat = 'rushing'
        field_list = table.columns
        DynamicItem = create_item_class(stat, field_list)
        item = DynamicItem()
        
        # yield items
        for record in table.to_dict(orient='record'):
            for k, v in record.items(): 
                item[k] = v
            yield item
            
        # Write out the game-by-game links 
        gamebygame = LinkExtractor(allow = self.gamebygameregex) 
        gamebygamelinks = gamebygame.extract_links(response)
        
        gamebygameItem = create_item_class('gamebygamelinks', ['team', 'year', 'link'])
        gameitem = gamebygameItem()
        
        for link in gamebygamelinks: 
            gameitem['team'] = response.meta['team']
            gameitem['year'] = year
            gameitem['link'] = link.url
            yield gameitem
            
        
        
        # Get all the links on the page
        le = LinkExtractor(restrict_xpaths = (self.xpath,),
                           allow = self.regex
                          ) 
        
        links = le.extract_links(response)
        
        # Build up the output 
        for link in links: 
            yield scrapy.Request(link.url, 
                                 callback=self.parse_stats,
                                 meta = {'year':year, 'team':response.meta['team'], 'stat':link.text}
                                )
    
    def parse_stats(self, response):
        print(response.meta['stat'], response.meta['year'], response.meta['team'])
        # Parse tables 
        table = self.table_cleaner(html = response.body, 
                                   target_table = 2, 
                                   stat = response.meta['stat'], 
                                   year = response.meta['year'], 
                                   team = response.meta['team'],
                                   trim_n_rows = 3
                                  )
        # Create dynamic items
        stat = response.meta['stat'].replace(" ","").replace("/","").replace(".","")
        field_list = table.columns
        DynamicItem = create_item_class(stat, field_list)
        item = DynamicItem()
        
        # yield items
        for record in table.to_dict(orient='record'):
            for k, v in record.items(): 
                item[k] = v
            yield item
    
    def table_cleaner(self, html, target_table, stat, year, team, trim_n_rows = 0):
        """Read HTML string and return a table
           html : HTML string
           target_table : Target table number
           trim_n_rows : Trim X number of rows from the end of the table (enter as positive number)
        """
        # Create tables
        tables = pd.read_html(html) 
        # Subset to single table of interest 
        table = tables[target_table] 
        # NCAA kicks out unlabeled columns for some reason. Find the first instance and use it to slice
        stop_index = [1 if 'Unnamed: ' in column  else 0 for idx, column in enumerate(table.columns)].index(1)
        # Fix the column names so no spaces, /'s, or .'s 
        columnNamesFixed = {col:col.replace(" ","").replace("/","").replace(".","") for col in table.iloc[:,:stop_index].columns}
        table.rename(columns = columnNamesFixed, inplace = True)
        # Clean up table 
        table = table[:-trim_n_rows].iloc[:,:stop_index]
        # Add static variables 
        table['stat'] = stat
        table['year'] = year
        table ['team'] = team
        return table 

In [5]:
process = CrawlerProcess({'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
                          'ITEM_PIPELINES': {'__main__.Pipeline':100}
                         })

process.crawl(StatsSpider)
process.start()

2017-12-11 14:42:51 [scrapy.utils.log] INFO: Scrapy 1.4.0 started (bot: scrapybot)
2017-12-11 14:42:51 [scrapy.utils.log] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2017-12-11 14:42:51 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2017-12-11 14:42:51 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHeadersMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.redirect.MetaRefreshMiddleware',
 'scrapy.downloadermiddlewares.httpcompression.HttpCompressionMiddleware',
 'sc

2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats/11520>
{'G': 12,
 'GP': '12',
 'GS': '7',
 'Jersey': '80',
 'Player': 'Griffin, Garrett',
 'Pos': 'TE',
 'RushAttempts': 2.0,
 'RushLong': '27',
 'RushNetYards': 27.0,
 'RushTDs': nan,
 'RushYdsG': 2.25,
 'RushYdsGained': '27',
 'RushYdsLost': nan,
 'YdsRush': 13.5,
 'Yr': 'So',
 'stat': 'rushing',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats/11520>
{'G': 12,
 'GP': '12',
 'GS': '0',
 'Jersey': '45',
 'Player': 'Fisher, Miles',
 'Pos': 'LB',
 'RushAttempts': nan,
 'RushLong': nan,
 'RushNetYards': nan,
 'RushTDs': nan,
 'RushYdsG': 0.0,
 'RushYdsGained': nan,
 'RushYdsLost': nan,
 'YdsRush': nan,
 'Yr': 'Jr',
 'stat': 'rushing',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/st

2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats/11520>
{'G': 11,
 'GP': '11',
 'GS': '0',
 'Jersey': '58',
 'Player': 'Ruechel, Andrew',
 'Pos': 'OL',
 'RushAttempts': nan,
 'RushLong': nan,
 'RushNetYards': nan,
 'RushTDs': nan,
 'RushYdsG': 0.0,
 'RushYdsGained': nan,
 'RushYdsLost': nan,
 'YdsRush': nan,
 'Yr': 'So',
 'stat': 'rushing',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats/11520>
{'G': 11,
 'GP': '11',
 'GS': '7',
 'Jersey': '15',
 'Player': 'Ludowig, Alex',
 'Pos': 'WR',
 'RushAttempts': nan,
 'RushLong': nan,
 'RushNetYards': nan,
 'RushTDs': nan,
 'RushYdsG': 0.0,
 'RushYdsGained': nan,
 'RushYdsLost': nan,
 'YdsRush': nan,
 'Yr': 'So',
 'stat': 'rushing',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats/11

2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats/11520>
{'G': 9,
 'GP': '9',
 'GS': '7',
 'Jersey': '96',
 'Player': 'Luebbe, Dana',
 'Pos': 'DL',
 'RushAttempts': nan,
 'RushLong': nan,
 'RushNetYards': nan,
 'RushTDs': nan,
 'RushYdsG': 0.0,
 'RushYdsGained': nan,
 'RushYdsLost': nan,
 'YdsRush': nan,
 'Yr': 'Sr',
 'stat': 'rushing',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats/11520>
{'G': 8,
 'GP': '8',
 'GS': '0',
 'Jersey': '6A',
 'Player': 'Steelhammer, Weston',
 'Pos': 'DB',
 'RushAttempts': nan,
 'RushLong': nan,
 'RushNetYards': nan,
 'RushTDs': nan,
 'RushYdsG': 0.0,
 'RushYdsGained': nan,
 'RushYdsLost': nan,
 'YdsRush': nan,
 'Yr': 'Fr',
 'stat': 'rushing',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats/115

2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats/11520>
{'G': 4,
 'GP': '4',
 'GS': '0',
 'Jersey': '26',
 'Player': 'Hancock, Brent',
 'Pos': 'WR',
 'RushAttempts': nan,
 'RushLong': nan,
 'RushNetYards': nan,
 'RushTDs': nan,
 'RushYdsG': 0.0,
 'RushYdsGained': nan,
 'RushYdsLost': nan,
 'YdsRush': nan,
 'Yr': 'Jr',
 'stat': 'rushing',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats/11520>
{'G': 4,
 'GP': '4',
 'GS': '3',
 'Jersey': '12',
 'Player': 'Awini, Jaleel',
 'Pos': 'QB',
 'RushAttempts': 45.0,
 'RushLong': '75',
 'RushNetYards': 220.0,
 'RushTDs': '4',
 'RushYdsG': 55.0,
 'RushYdsGained': '250',
 'RushYdsLost': 30.0,
 'YdsRush': 4.89,
 'Yr': 'So',
 'stat': 'rushing',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:42:53 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stat

...

2017-12-11 14:43:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11520&year_stat_category_id=10395>
{'APY': 0,
 'All-PurposeYardsG': 0.0,
 'FRetYds': nan,
 'G': 1,
 'GP': '1',
 'GS': '0',
 'IntRYds': nan,
 'Jersey': '67',
 'KORetYds': nan,
 'Player': 'Njemanze, Jabree',
 'Pos': 'OL',
 'PuntRetYds': nan,
 'ReceivingYards': nan,
 'RushNetYards': nan,
 'Yr': 'So',
 'stat': 'All-Purpose Yards',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:43:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11520&year_stat_category_id=10395>
{'APY': 0,
 'All-PurposeYardsG': 0.0,
 'FRetYds': nan,
 'G': 1,
 'GP': '1',
 'GS': '0',
 'IntRYds': nan,
 'Jersey': '54',
 'KORetYds': nan,
 'Player': 'Bungum, Dylan',
 'Pos': 'DL',
 'PuntRetYds': nan,
 'ReceivingYards': nan,
 'RushNetYards': nan,
 'Yr': 'Jr',
 'stat': 'All-Purpose Yards',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:43:05 [scrapy.cor

2017-12-11 14:43:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11520&year_stat_category_id=10394>
{'FirstDownsbyPenalty': nan,
 'G': 2,
 'GP': '2',
 'GS': '0',
 'Jersey': '9B',
 'PassYards': nan,
 'PassingFirstDowns': nan,
 'Penalties': nan,
 'PenaltiesPerGame': nan,
 'PenaltyYards': nan,
 'PenaltyYdsPerGame': nan,
 'Player': 'Oehrle, Drew',
 'Plays': nan,
 'Pos': 'K',
 'RushNetYards': nan,
 'RushingFirstDowns': nan,
 'TOP': nan,
 'TotOff': 0,
 'TotalOffYardsG': 0.0,
 'YdsPlay': nan,
 'Yr': 'So',
 'stat': 'Total Offense',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:43:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11520&year_stat_category_id=10394>
{'FirstDownsbyPenalty': nan,
 'G': 2,
 'GP': '2',
 'GS': '0',
 'Jersey': '70',
 'PassYards': nan,
 'PassingFirstDowns': nan,
 'Penalties': nan,
 'PenaltiesPerGame': nan,
 'PenaltyYards': nan,
 'PenaltyYdsPerGame': nan,
 'Player': 'Uzo-O

2017-12-11 14:43:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11520&year_stat_category_id=10396>
{'FG': nan,
 'FumbRetPAT': nan,
 'G': 1,
 'GP': '1',
 'GS': '0',
 'Jersey': '57',
 'KickPAT': nan,
 'KickReturnPAT': nan,
 'PATAtt': nan,
 'Pass2PTAtt': nan,
 'Player': 'Cooksey, Taylor',
 'Points': 0,
 'PointsPerGame': 0.0,
 'Pos': 'OL',
 'ReceivingPAT': nan,
 'Ru2PTAtt': nan,
 'RushPAT': nan,
 'Safeties': nan,
 'TDs': nan,
 'Yr': 'Jr',
 'stat': 'Scoring',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:43:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11520&year_stat_category_id=10396>
{'FG': nan,
 'FumbRetPAT': nan,
 'G': 1,
 'GP': '1',
 'GS': '0',
 'Jersey': '84',
 'KickPAT': nan,
 'KickReturnPAT': nan,
 'PATAtt': nan,
 'Pass2PTAtt': nan,
 'Player': 'Onwuzu, Nelson',
 'Points': 0,
 'PointsPerGame': 0.0,
 'Pos': 'WR',
 'ReceivingPAT': nan,
 'Ru2PTAtt': nan,
 'RushPAT': nan,
 'Safeties'

 'year': '2013-14'}
2017-12-11 14:43:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11520&year_stat_category_id=10393>
{'G': 1,
 'GP': '1',
 'GS': '0',
 'Jersey': '10',
 'Player': 'Gann, Christian',
 'Pos': 'WR',
 'Rec': nan,
 'RecTD': nan,
 'RecYdsG': 0.0,
 'ReceivingYards': nan,
 'ReceptionsPerGame': 0.0,
 'YardsPerReception': nan,
 'Yr': 'Jr',
 'stat': 'Receiving',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:43:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11520&year_stat_category_id=10393>
{'G': 1,
 'GP': '1',
 'GS': '0',
 'Jersey': '73',
 'Player': 'Holba, Andrew',
 'Pos': 'OL',
 'Rec': nan,
 'RecTD': nan,
 'RecYdsG': 0.0,
 'ReceivingYards': nan,
 'ReceptionsPerGame': 0.0,
 'YardsPerReception': nan,
 'Yr': 'So',
 'stat': 'Receiving',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:43:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/

2017-12-11 14:43:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11520&year_stat_category_id=10396>
{'FG': nan,
 'FumbRetPAT': nan,
 'G': 1,
 'GP': '1',
 'GS': '0',
 'Jersey': '52',
 'KickPAT': nan,
 'KickReturnPAT': nan,
 'PATAtt': nan,
 'Pass2PTAtt': nan,
 'Player': 'Salimu, Yohance',
 'Points': 0,
 'PointsPerGame': 0.0,
 'Pos': 'DL',
 'ReceivingPAT': nan,
 'Ru2PTAtt': nan,
 'RushPAT': nan,
 'Safeties': nan,
 'TDs': nan,
 'Yr': 'So',
 'stat': 'Scoring',
 'team': 'Air Force Falcons',
 'year': '2013-14'}
2017-12-11 14:43:05 [scrapy.core.scraper] DEBUG: Scraped from <200 http://stats.ncaa.org/team/721/stats?id=11520&year_stat_category_id=10396>
{'FG': nan,
 'FumbRetPAT': nan,
 'G': 1,
 'GP': '1',
 'GS': '0',
 'Jersey': '8A',
 'KickPAT': nan,
 'KickReturnPAT': nan,
 'PATAtt': nan,
 'Pass2PTAtt': nan,
 'Player': 'Davis, Pate',
 'Points': 0,
 'PointsPerGame': 0.0,
 'Pos': nan,
 'ReceivingPAT': nan,
 'Ru2PTAtt': nan,
 'RushPAT': nan,
 'Safeties': na